In [ ]:
import pandas as pd
import numpy as np

# [Step 1] 원본 데이터 로드 및 복사
customer = pd.read_csv("customer_hm.csv")
transactions = pd.read_csv("transactions_hm.csv")
articles = pd.read_csv("articles_hm.csv")

# 경로에 맞게 위 아래 중 선택 주석처리
# customer = pd.read_csv('../../data/customer_hm.csv')
# transactions = pd.read_csv('../../data/transactions_hm.csv')
# articles = pd.read_csv('../../data/articles_hm.csv')

#copy
df_cust = customer.copy()
df_tran = transactions.copy()
df_art = articles.copy()

# [Customer] 데이터 전처리

#1. fashion_news_frequency가 결측치인 데이터만 추출 (결측치 Regularly로 대체)
missing_data = df_cust[df_cust['fashion_news_frequency'].isnull()]

#1-2. 결측치 'Regularly'으로 채우기
df_cust['fashion_news_frequency'] = df_cust['fashion_news_frequency'].fillna('Regularly')

#2. 문자형 정수형으로 매핑 (사전 정의)
status_map = {'ACTIVE': 2, 'PRE-CREATE': 1, 'LEFT CLUB': 0}
frequency_map = {'Regularly': 2, 'Monthly': 1, 'NONE': 0}

#2-2.  매핑 적용
df_cust['club_member_status'] = df_cust['club_member_status'].map(status_map)
df_cust['fashion_news_frequency'] = df_cust['fashion_news_frequency'].map(frequency_map)


#3. 연령대 그룹화 (age_segment) : 나이를 나이대별로 나눠 파생변수 생성
def cate_age(age):
    if age < 20: return '10대'
    elif age < 30: return '20대'
    elif age < 40: return '30대'
    elif age < 50: return '40대'
    elif age < 60: return'50대'
    else: return '60대 이상'

df_cust['age_segment'] = df_cust['age'].apply(cate_age)


# [Transactions] 데이터 전처리

#1. join을 위해 동일한 타입으로 변환
df_tran['customer_id']= df_tran['customer_id'].astype(str)
df_cust['customer_id']= df_cust['customer_id'].astype(str)

df_tran['article_id']= df_tran['article_id'].astype(str).str.zfill(10)
df_art['article_id'] = df_art['article_id'].astype(str).str.zfill(10)

#2. 데이터 크기 및 중복 제거
df_tran.drop_duplicates(inplace=True)

#3. 날짜 타입 변환 및 월 컬럼 생성
df_tran['t_dat'] = pd.to_datetime(df_tran['t_dat'], format='%Y-%m-%d')
df_tran['year_month'] = df_tran['t_dat'].dt.to_period('M')

#4. 판매 채널 한글 컬럼 생성 및 값 변환
df_tran["channel"] = df_tran["sales_channel_id"].map({1: "오프라인", 2: "온라인"})

df_tran["channel"].value_counts() 
numbs = df_tran["channel"].value_counts()
print("온라인:", f"{numbs['온라인']:,}")
print("오프라인:", f"{numbs['오프라인']:,}")
print("전체 거래 건수:", f"{df_tran.shape[0]:,}")

#가격 데이터 분포 확인
df_tran['price'].describe() 
print("price가 0.1 이상:", len(df_tran[df_tran["price"] >= 0.1]))
print("price가 0.2 이상:", len(df_tran[df_tran["price"] >= 0.2]))
print("price가 0.3 이상:", len(df_tran[df_tran["price"] >= 0.3]))
print("price가 0.4 이상:", len(df_tran[df_tran["price"] >= 0.4]))
print("price가 0.5 이상:", len(df_tran[df_tran["price"] >= 0.5]))

#고가 거래 분리 및 채널 확인
high_price_transactions = df_tran[df_tran["price"] >= 0.4]
high_price_transactions['sales_channel_id'].replace({1: 'Offline', 2: 'Online'}).value_counts() 

#월별 거래 건수 & 매출액 계산
monthly_sales = df_tran.groupby('year_month').size()
monthly_price = df_tran.groupby('year_month')['price'].sum()

print(monthly_price)

#최고 매출 월 
best_month = monthly_price.idxmax()
best_value = monthly_price.max()

print("가장 매출이 높은 달:", best_month)
print("그 달의 매출:", best_value) 


# [Article] 데이터 전처리: 상품 전략 파생변수 생성

#1. ID 형식 통일
df_art['article_id'] = df_art['article_id'].astype(str).str.zfill(10)

#2. 결측치(NULL) 제어 (detail_desc의 NULL값을 No Description로)
df_art['detail_desc'] = df_art['detail_desc'].fillna('No Description')

#3. 불필요 컬럼 제거 (메모리 줄이기) : Name(이름)이 존재하여 중복 정보인 Code(코드성 숫자) 컬럼 불필요
cols_to_drop = [
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id',
    'department_no',
    'index_code',
    'index_group_no',
    'section_no',
    'garment_group_no'
]
df_art_cleaned = df_art.drop(columns=cols_to_drop) 

#4. 매핑 및 파생변수 설정
#4-1. 시즌 별 매핑 (product_season) / 시즌성 구분을 위한 키워드 정의
def get_season(section):
    section = section.lower()
    # FW
    if any(kw in section for kw in ['outerwear', 'nightwear', 'socks', 'tights', 'knitted']):
        return 'FW'
    # SS
    elif any(kw in section for kw in ['swimwear', 'sport', 'shorts', 'sandals']):
        return 'SS'
    # 두루두루 아이템(all season)
    else:
        return 'All-Season'

#4-2. 파생변수 생성 및 이름만 조금 더 직관적으로 분류
df_art_cleaned['product_season'] = df_art_cleaned['section_name'].apply(get_season)
#4-3. 메인 카테고리 정의 (원본 보존)
df_art_cleaned['category_main'] = df_art_cleaned['index_group_name']   


#5. 저관여/고관여 제품 분류 (product_strategy)
def get_involvement_strategy(row):
    garment = str(row['garment_group_name']).lower()
    section = str(row['section_name']).lower()
    # 1. 저관여 (베이직)
    if any(kw in garment for kw in ['basic', 'underwear', 'socks', 'jersey']) or \
       'basic' in section:
        return 'Low_Involvement_Basic'
    # 2. 고관여 (전략템/트렌드템)
    elif any(kw in garment for kw in ['knitwear', 'outerwear', 'dresses']) or \
         any(kw in section for kw in ['trend', 'special']):
        return 'High_Involvement_Strategic'
    else:
        return 'General_Fashion'
    
df_art_cleaned['product_strategy'] = df_art_cleaned.apply(get_involvement_strategy, axis=1)

#6. 신상품/트렌드 키워드 추출 (is_new)
def check_newness(row):
    text = (str(row['prod_name']) + " " + str(row['detail_desc'])).lower()
    new_keywords = ['new', 'collection', 'latest', 'trend', 'exclusive']
    
    if any(kw in text for kw in new_keywords):
        return 'New_Arrival'
    return 'Regular_Carryover'

df_art_cleaned['is_new'] = df_art_cleaned.apply(check_newness, axis=1)

#7. 색상 그룹화 (color_tone)
def get_color_tone(color):
    color = str(color).lower()
    dark_colors = ['black', 'dark blue', 'dark grey', 'dark red', 'navy blue', 'dark green', 'anthracite']
    light_colors = ['white', 'light beige', 'off white', 'light pink', 'light blue', 'yellowish brown']
    
    if any(dc in color for dc in dark_colors):
        return 'Dark_Tone'
    elif any(lc in color for lc in light_colors):
        return 'Light_Tone'
    else:
        return 'Neutral_Tone'

df_art_cleaned['color_tone'] = df_art_cleaned['colour_group_name'].apply(get_color_tone)

온라인: 721,488
오프라인: 318,613
전체 거래 건수: 1,040,101
price가 0.1 이상: 10704
price가 0.2 이상: 719
price가 0.3 이상: 126
price가 0.4 이상: 29
price가 0.5 이상: 2
year_month
2019-01    2129.926131
2019-02    1989.217641
2019-03    2374.905504
2019-04    2703.443538
2019-05    2748.199469
2019-06    3088.776976
2019-07    2552.035334
2019-08    1943.422489
2019-09    2559.226862
2019-10    2358.486793
2019-11    2463.769270
2019-12    1985.492149
Freq: M, Name: price, dtype: float64
가장 매출이 높은 달: 2019-06
그 달의 매출: 3088.776976142


In [3]:
##체크 용도
#df_cust
display(df_cust[['customer_id', 'age', 'age_segment', 'club_member_status', 'fashion_news_frequency']].head())

#df_tran
display(df_tran[['customer_id', 'article_id', 't_dat', 'year_month', 'channel', 'price']].head())

#df_art_cleaned
display(df_art_cleaned[['article_id', 'prod_name', 'product_strategy', 'is_new', 'product_season', 'color_tone']].head())

,customer_id,age,age_segment,club_member_status,fashion_news_frequency
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,49,40대,2,0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,25,20대,2,0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24,20대,2,0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,54,50대,2,0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,52,50대,2,2


,customer_id,article_id,t_dat,year_month,channel,price
0,3e2b60b679e62fb49516105b975560082922011dd752ec...,0698328010,2019-11-05,2019-11,온라인,0.016932
1,89647ac2274f54c770aaa4b326e0eea09610c252381f37...,0760597002,2019-05-22,2019-05,온라인,0.033881
2,2ebe392150feb60ca89caa8eff6c08b7ef1138cd6fdc71...,0488561032,2019-05-10,2019-05,온라인,0.016932
3,7b3205de4ca17a339624eb5e3086698e9984eba6b47c56...,0682771001,2019-08-26,2019-08,온라인,0.033881
4,3b77905de8b32045f08cedb79200cdfa477e9562429a39...,0742400033,2019-08-10,2019-08,오프라인,0.003220


,article_id,prod_name,product_strategy,is_new,product_season,color_tone
0,0108775015,Strap top,Low_Involvement_Basic,Regular_Carryover,All-Season,Dark_Tone
1,0108775044,Strap top,Low_Involvement_Basic,Regular_Carryover,All-Season,Light_Tone
2,0108775051,Strap top (1),Low_Involvement_Basic,Regular_Carryover,All-Season,Light_Tone
3,0110065001,OP T-shirt (Idro),General_Fashion,Regular_Carryover,All-Season,Dark_Tone
4,0110065002,OP T-shirt (Idro),General_Fashion,Regular_Carryover,All-Season,Light_Tone


### [Data Dictionary] 

#### 1. 파생변수
- **product_strategy** : 상품 관여도 전략 (Basic / Strategic / General)
- **age_segment** : 고객 연령대 세그먼트 (10대 ~ 60대+)
- **is_new** : 신상품 반응 지표 (New_Arrival / Regular_Carryover)
- **product_season** : 상품 시즌 속성 (FW / SS / All-Season)
- **color_tone** : 디자인 톤앤매너 (Dark / Light / Neutral)

#### 2. 기본 정보 (Raw Data)
- **channel** : 온/오프라인 구매 채널 (매출 발생 지점 확인)
- **club_member_status** : 멤버십 등급 (로열티 고객의 구매 유지력 확인)
- **fashion_news_freq** : 마케팅 뉴스레터 구독 빈도 (CRM 활동 영향력 측정)
- **price** : 상품 판매 단가 (객단가 및 가격대별 수요 분석)
- **year_month** : 시간 지표 (6월 정점 대비 8월 하락세의 기준축)
- **category_main** : 여성/남성/아동 등 메인 카테고리 (품목별 성과 측정)

In [ ]:
# [JOIN] df_tran 기준 Left Join
# 고객의 나이대(age_segment), 뉴스 구독 빈도 등을 거래 내역에 연결
df_merged = pd.merge(
    df_tran, 
    df_cust, 
    on='customer_id', 
    how='left'
)

# articles 조인은 이후 필요하면 진행

조인 후 전체 행 수: 1040101
고객 정보가 매칭되지 않은 거래 건수: 233994
